In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
arg_date = '2021-05-07'

In [3]:
arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

In [4]:
arg_date_dt

datetime.date(2021, 5, 6)

In [5]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('deutsche-boerse-xetra-pds')

In [6]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], '%Y-%m-%d').date() >= arg_date_dt]

In [7]:
objects

[s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pds', key='2021-05-06/2021-05-06_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='deutsche-boerse-xetra-pd

In [8]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=',')

In [9]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [10]:
df_all = pd.DataFrame(columns=df_init.columns)
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    df_all = df_all.append(df, ignore_index=True)

In [11]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2021-05-06,07:00,21.52,21.90,21.52,21.82,2648,10
1,DE000A0DJ6J9,S92,SMA SOLAR TECHNOL.AG,Common stock,EUR,2504287,2021-05-06,07:00,46.02,46.02,45.82,45.84,1766,10
2,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2021-05-06,07:00,21.56,21.78,21.56,21.78,14935,14
3,DE000A0D9PT0,MTX,MTU AERO ENGINES NA O.N.,Common stock,EUR,2504297,2021-05-06,07:00,203.60,203.60,202.50,202.50,1246,9
4,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2021-05-06,07:00,44.18,44.21,44.04,44.08,12002,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225662,NL0010273215,ASME,"ASML HOLDING EO -,09",Common stock,EUR,2506262,2021-05-10,07:52,535.10,535.10,535.10,535.10,2,1
225663,US0231351067,AMZ,"AMAZON.COM INC. DL-,01",Common stock,EUR,2506427,2021-05-10,07:52,2704.00,2704.00,2704.00,2704.00,20,1
225664,DE0006452907,NEM,NEMETSCHEK SE O.N.,Common stock,EUR,2505040,2021-05-10,07:52,57.06,57.08,57.06,57.08,231,5
225665,DE0005089031,UTDI,UTD.INTERNET AG NA,Common stock,EUR,2504878,2021-05-10,07:52,35.43,35.44,35.43,35.44,763,2


In [12]:
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
df_all = df_all.loc[:, columns]

In [13]:
df_all

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume
0,AT0000A0E9W5,2021-05-06,07:00,21.52,21.90,21.52,21.82,2648
1,DE000A0DJ6J9,2021-05-06,07:00,46.02,46.02,45.82,45.84,1766
2,DE000A0D6554,2021-05-06,07:00,21.56,21.78,21.56,21.78,14935
3,DE000A0D9PT0,2021-05-06,07:00,203.60,203.60,202.50,202.50,1246
4,DE000A0HN5C6,2021-05-06,07:00,44.18,44.21,44.04,44.08,12002
...,...,...,...,...,...,...,...,...
225662,NL0010273215,2021-05-10,07:52,535.10,535.10,535.10,535.10,2
225663,US0231351067,2021-05-10,07:52,2704.00,2704.00,2704.00,2704.00,20
225664,DE0006452907,2021-05-10,07:52,57.06,57.08,57.06,57.08,231
225665,DE0005089031,2021-05-10,07:52,35.43,35.44,35.43,35.44,763


In [14]:
df_all.dropna(inplace=True)

In [15]:
df_all.shape

(225667, 8)

## Get opening price per ISIN and day

In [16]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')

In [17]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price
0,AT0000A0E9W5,2021-05-06,07:00,21.52,21.90,21.52,21.82,2648,21.52
151,AT0000A0E9W5,2021-05-06,07:01,21.90,21.90,21.78,21.78,373,21.52
273,AT0000A0E9W5,2021-05-06,07:02,21.76,22.04,21.76,22.04,9534,21.52
673,AT0000A0E9W5,2021-05-06,07:04,22.00,22.04,22.00,22.00,8376,21.52
2845,AT0000A0E9W5,2021-05-06,07:05,22.00,22.06,22.00,22.04,2002,21.52
...,...,...,...,...,...,...,...,...,...
223843,AT0000A0E9W5,2021-05-10,07:43,21.70,21.70,21.70,21.70,63,21.34
223960,AT0000A0E9W5,2021-05-10,07:44,21.70,21.72,21.66,21.66,1868,21.34
224024,AT0000A0E9W5,2021-05-10,07:45,21.64,21.64,21.62,21.64,1261,21.34
224953,AT0000A0E9W5,2021-05-10,07:49,21.60,21.60,21.60,21.60,1540,21.34


## Get closing price per ISIN and day

In [18]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')

In [19]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,opening_price,closing_price
0,AT0000A0E9W5,2021-05-06,07:00,21.52,21.90,21.52,21.82,2648,21.52,21.50
151,AT0000A0E9W5,2021-05-06,07:01,21.90,21.90,21.78,21.78,373,21.52,21.50
273,AT0000A0E9W5,2021-05-06,07:02,21.76,22.04,21.76,22.04,9534,21.52,21.50
673,AT0000A0E9W5,2021-05-06,07:04,22.00,22.04,22.00,22.00,8376,21.52,21.50
2845,AT0000A0E9W5,2021-05-06,07:05,22.00,22.06,22.00,22.04,2002,21.52,21.50
...,...,...,...,...,...,...,...,...,...,...
223843,AT0000A0E9W5,2021-05-10,07:43,21.70,21.70,21.70,21.70,63,21.34,21.62
223960,AT0000A0E9W5,2021-05-10,07:44,21.70,21.72,21.66,21.66,1868,21.34,21.62
224024,AT0000A0E9W5,2021-05-10,07:45,21.64,21.64,21.62,21.64,1261,21.34,21.62
224953,AT0000A0E9W5,2021-05-10,07:49,21.60,21.60,21.60,21.60,1540,21.34,21.62


## Aggregations

In [20]:
df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))

In [21]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume
0,AT00000FACC2,2021-05-06,8.880,8.730,8.730,8.880,943
1,AT00000FACC2,2021-05-07,8.860,9.000,8.840,9.000,547
2,AT00000FACC2,2021-05-10,9.100,8.940,8.940,9.100,76
3,AT0000606306,2021-05-06,18.500,18.300,18.300,18.540,9273
4,AT0000606306,2021-05-07,18.650,18.740,18.600,18.740,1283
...,...,...,...,...,...,...,...
8482,XS2265370234,2021-05-07,24.606,24.108,23.766,24.606,1848
8483,XS2265370234,2021-05-10,24.234,24.234,24.234,24.234,0
8484,XS2284324667,2021-05-06,26.744,26.686,26.586,26.744,2721
8485,XS2284324667,2021-05-07,26.680,26.658,26.430,26.680,5211


## Percent Change Prev Closing

In [22]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [23]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,prev_closing_price
0,AT00000FACC2,2021-05-06,8.880,8.730,8.730,8.880,943,NaN
1,AT00000FACC2,2021-05-07,8.860,9.000,8.840,9.000,547,8.730
2,AT00000FACC2,2021-05-10,9.100,8.940,8.940,9.100,76,9.000
3,AT0000606306,2021-05-06,18.500,18.300,18.300,18.540,9273,NaN
4,AT0000606306,2021-05-07,18.650,18.740,18.600,18.740,1283,18.300
...,...,...,...,...,...,...,...,...
8482,XS2265370234,2021-05-07,24.606,24.108,23.766,24.606,1848,24.446
8483,XS2265370234,2021-05-10,24.234,24.234,24.234,24.234,0,24.108
8484,XS2284324667,2021-05-06,26.744,26.686,26.586,26.744,2721,NaN
8485,XS2284324667,2021-05-07,26.680,26.658,26.430,26.680,5211,26.686


In [24]:
df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100

In [25]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [26]:
df_all = df_all.round(decimals=2)

In [27]:
df_all = df_all[df_all.Date >= arg_date]

In [28]:
df_all

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
1,AT00000FACC2,2021-05-07,8.86,9.00,8.84,9.00,547,3.09
2,AT00000FACC2,2021-05-10,9.10,8.94,8.94,9.10,76,-0.67
4,AT0000606306,2021-05-07,18.65,18.74,18.60,18.74,1283,2.40
5,AT0000606306,2021-05-10,18.75,18.75,18.68,18.75,42,0.05
7,AT0000609607,2021-05-07,16.42,16.62,16.42,16.70,95,1.71
...,...,...,...,...,...,...,...,...
8480,XS2265369731,2021-05-10,10.40,10.40,10.40,10.40,0,0.58
8482,XS2265370234,2021-05-07,24.61,24.11,23.77,24.61,1848,-1.38
8483,XS2265370234,2021-05-10,24.23,24.23,24.23,24.23,0,0.52
8485,XS2284324667,2021-05-07,26.68,26.66,26.43,26.68,5211,-0.10


## Write to S3

In [29]:
key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'

In [30]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket('xetra-1234')
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='xetra-1234', key='xetra_daily_report_20210510_101629.parquet')

## Reading the uploaded file

In [31]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20210510_101629.parquet


In [33]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20210510_101629.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [34]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT00000FACC2,2021-05-07,8.86,9.00,8.84,9.00,547,3.09
1,AT00000FACC2,2021-05-10,9.10,8.94,8.94,9.10,76,-0.67
2,AT0000606306,2021-05-07,18.65,18.74,18.60,18.74,1283,2.40
3,AT0000606306,2021-05-10,18.75,18.75,18.68,18.75,42,0.05
4,AT0000609607,2021-05-07,16.42,16.62,16.42,16.70,95,1.71
...,...,...,...,...,...,...,...,...
5503,XS2265369731,2021-05-10,10.40,10.40,10.40,10.40,0,0.58
5504,XS2265370234,2021-05-07,24.61,24.11,23.77,24.61,1848,-1.38
5505,XS2265370234,2021-05-10,24.23,24.23,24.23,24.23,0,0.52
5506,XS2284324667,2021-05-07,26.68,26.66,26.43,26.68,5211,-0.10
